In [285]:
import psycopg2
import pandas as pd
import re
from io import StringIO
import os
import numpy as np
import string
import csv

In [286]:
def create_database():
    #Connect to the default database
    try:
        conn = psycopg2.connect("""host=localhost dbname=postgres user=postgres password=''""")
        conn.set_session(autocommit=True)
    except psycopg2.Error as e:
        print("Exception while connecting to database!")
        print(f"Error: {e}")
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("""Error while creating the cursor!""")
        print(f"Error: {e}")
        
    # creating database
#     cur.execute("DROP DATABASE pokemon_db")
#     cur.execute("CREATE DATABASE pokemon_db")
    
    # close default connection
    conn.close()
    
    # connect to accounts database
    try:
        conn = psycopg2.connect("""host=localhost dbname=pokemon_db user=postgres password=''""")
        cur = conn.cursor()
    except pyscopg2.Error as e:
        print("Error while connecting to the database")
        print(f"Error: {e}")
    
    return cur, conn
    

In [287]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [288]:
def create_tables(curr, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [289]:
def insert_data(conn, df, table):
    """Save dataframe into memory and use copy_from() to copy data to the table"""
    ## Save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer,index=False, header=False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=',')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("Insert complete")
    cursor.close()
        

In [290]:
import string
PokemonAbilities = pd.read_csv("data/gen9_pokemon_abilities.csv",encoding='latin-1')
clean_columns = list(map(lambda x:re.sub(r'[^\w\s]','',x.lower().replace(" ","_")), PokemonAbilities.columns.to_list()))
PokemonAbilities.columns = clean_columns
chars = '"-,.'
PokemonAbilities['games_text'] = PokemonAbilities['games_text'].str.translate(str.maketrans('','', chars))
PokemonAbilities['indepth_effect'] = PokemonAbilities['indepth_effect'].str.translate(str.maketrans('','', chars))
PokemonAbilities['id'] = np.arange(1,PokemonAbilities.shape[0]+1)
PokemonAbilities.fillna("", inplace=True)
PokemonAbilities = PokemonAbilities[['id','ability','pokemon','games_text','indepth_effect','ability_url']]
PokemonAbilities.head()

ParserError: Error tokenizing data. C error: Expected 8 fields in line 30, saw 10


In [291]:
PokemonAbilities['pokemon'].duplicated().any()

True

In [292]:
PokemonStats= pd.read_csv("data/gen9_pokemon_stats.csv")
clean_columns = list(map(lambda x:re.sub(r'[^\w\s]','_',x.lower().replace(" ","_")), PokemonStats.columns.to_list()))
PokemonStats.columns = clean_columns
PokemonStats = PokemonStats[['pokemon','type_1','type_2','hp','attack','defense','special_attack','special_defense','speed','weakness_normal','weakness_fire','weakness_water','weakness_electric']]
PokemonStats.fillna("", inplace=True)
PokemonStats.head()

,pokemon,type_1,type_2,hp,attack,defense,special_attack,special_defense,speed,weakness_normal,weakness_fire,weakness_water,weakness_electric
0,Annihilape,fighting,ghost,110,115,80,50,90,90,0.0,1.0,1.0,1.0
1,Arboliva,grass,normal,78,69,90,125,109,39,1.0,2.0,0.5,0.5
2,Arctibax,ice,dragon,90,95,66,45,65,62,1.0,1.0,0.5,0.5
3,Armarouge,fire,psychic,85,60,100,125,80,75,1.0,0.5,2.0,1.0
4,Baxcalibur,ice,dragon,115,145,92,75,86,87,1.0,1.0,0.5,0.5


In [293]:
PokemonStats['pokemon'].duplicated().any()

False

In [294]:
PokemonMoves= pd.read_csv("data/gen9_pokemon_moves.csv",encoding='latin-1')
clean_columns = list(map(lambda x:re.sub(r'[^\w\s]','_',x.lower().replace(" ","_")), PokemonMoves.columns.to_list()))
PokemonMoves.columns = clean_columns
PokemonMoves['id'] = np.arange(1,PokemonMoves.shape[0]+1)
PokemonMoves = PokemonMoves[['id','move','type','category','power_points','base_power','accuracy','battle_effect','secondary_effect','secondary_rate','critical_rate','speed_priority','pokemon']]
# chars = '"-,.'
# PokemonMovesStrCols['battle_effect'] = PokemonMoves['battle_effect'].str.replace('""')
# # PokemonMoves[['battle_effect','secondary_effect']].applymap(lambda x:x.replace('"',''))
# PokemonMovesStrCols.to_csv("moves.csv")
PokemonMoves.fillna("", inplace=True)
PokemonMoves.head()

,id,move,type,category,power_points,base_power,accuracy,battle_effect,secondary_effect,secondary_rate,critical_rate,speed_priority,pokemon
0,1,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.,0.0,0.0417,0,Smoliv
1,2,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.,0.0,0.0417,0,Dolliv
2,3,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.,0.0,0.0417,0,Arboliva
3,4,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.,0.0,0.0417,0,Bramblin
4,5,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.,0.0,0.0417,0,Brambleghast


In [295]:
PokemonMoves= pd.read_csv("data/gen9_pokemon_moves.csv",encoding='latin-1')
clean_columns = list(map(lambda x:re.sub(r'[^\w\s]','_',x.lower().replace(" ","_")), PokemonMoves.columns.to_list()))
PokemonMoves.columns = clean_columns
PokemonMoves['id'] = np.arange(1,PokemonMoves.shape[0]+1)
PokemonMoves = PokemonMoves[['id','move','type','category','power_points','base_power','accuracy','secondary_rate','critical_rate','speed_priority','pokemon']]
PokemonMoves.to_csv("moves.csv")
PokemonMoves.head()

,id,move,type,category,power_points,base_power,accuracy,secondary_rate,critical_rate,speed_priority,pokemon
0,1,Absorb,grass,special,25,20,100,0.0,0.0417,0,Smoliv
1,2,Absorb,grass,special,25,20,100,0.0,0.0417,0,Dolliv
2,3,Absorb,grass,special,25,20,100,0.0,0.0417,0,Arboliva
3,4,Absorb,grass,special,25,20,100,0.0,0.0417,0,Bramblin
4,5,Absorb,grass,special,25,20,100,0.0,0.0417,0,Brambleghast


In [296]:
cur, conn = create_database()
create_pokemon_stats_table = ("""
CREATE TABLE IF NOT EXISTS pokemon_stats
(
    pokemon character varying NOT NULL,
    type_1 character varying,
    type_2 character varying,
    hp integer,
    attack integer,
    defense integer,
    special_attack integer,
    special_defense integer,
    speed integer,
    weakness_normal double precision,
    weakness_fire double precision,
    weakness_water double precision,
    weakness_electric double precision,
    PRIMARY KEY (pokemon)
);
""")

In [297]:
cur.execute(create_pokemon_stats_table)
conn.commit()

In [298]:
create_pokemon_moves_table = ("""
CREATE TABLE IF NOT EXISTS pokemon_moves
(
    move_id serial PRIMARY KEY NOT NULL,
    move character varying,
    type character varying,
    category character varying,
    power_points double precision,
    base_power double precision,
    accuracy double precision,
    secondary_rate double precision,
    critical_rate double precision,
    speed_priority integer,
    pokemon character varying
);
""")

In [299]:
cur.execute(create_pokemon_moves_table)
conn.commit()

In [300]:
create_pokemon_abilities_table = ("""
CREATE TABLE IF NOT EXISTS pokemon_abilities
(
    ability_id serial PRIMARY KEY NOT NULL,
    ability character varying,
    pokemon character varying,
    game_text character varying,
    indepth_effect character varying,
    ability_url character varying
);

""")


In [301]:
cur.execute(create_pokemon_abilities_table)
conn.commit()

In [302]:
## Insert Into pokemon_stats
insert_data(conn, PokemonStats, "pokemon_stats")

Insert complete


In [303]:
## Insert into pokemon moves
insert_data(conn, PokemonMoves, "pokemon_moves")

Insert complete


In [304]:
## Insert into pokemon moves
insert_data(conn, PokemonAbilities, "pokemon_abilities")

Insert complete
